In [1]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import datetime
import warnings
import time
from scipy import stats
from sklearn import linear_model
import itertools
import calendar
warnings.filterwarnings('ignore')

In [2]:
data_sample = pd.read_csv('20171016-210106-DataSample.csv',dtype={"value": float})
data_sample2 = pd.read_csv('20171016-210304-DataSample.csv',dtype={"value": float})
data_sample3 = pd.read_csv('20171016-210529-DataSample.csv',dtype={"value": float})
data_sample4 = pd.read_csv('20171031-235959-DataSample.csv',dtype={"value": float})



data_sample = data_sample.append([data_sample2,data_sample3,data_sample4])
data_sample['startDate'] = pd.to_datetime(data_sample['startDate']) 
data_sample['owner'].replace('00000000-5854-8d6f-b8eb-cf14a0f795df','00000000-56ff-538b-2223-e1800b5e3ddb',inplace=True)
data_sample['startDate'] = pd.to_datetime(data_sample['startDate'])
data_sample['endDate'] = pd.to_datetime(data_sample['endDate'])
data_sample.index = data_sample['startDate']

to_tdelta = lambda row: row['endDate'] - row['startDate']
data_sample['duration'] = data_sample.apply(to_tdelta, axis=1)
data_sample['day_of_week'] = data_sample['startDate'].dt.dayofweek
data_sample.sort_index(inplace=True)
data_sample.head()

,_id,owner,source,sourceId,sourceName,type,startDate,endDate,value,content,originalData,createdAt,updatedAt,duration,day_of_week
startDate,,,,,,,,,,,,,,,
2012-01-19 07:49:00,36e3fbf8-53bd-4f17-ae4c-314f59aab97e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 18:05:32.162199-07,2017-10-12 15:09:13.597742-07,00:02:00,3
2012-01-19 07:49:00,36e3fbf8-53bd-4f17-ae4c-314f59aab97e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 18:05:32.162199-07,2017-10-12 15:09:13.597742-07,00:02:00,3
2012-01-19 07:49:00,dee37862-6869-488e-b301-c21bc69e6f1e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 11:07:13.318881-07,2017-10-11 17:32:01.100877-07,00:02:00,3
2012-01-19 07:49:00,36e3fbf8-53bd-4f17-ae4c-314f59aab97e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 18:05:32.162199-07,2017-10-12 15:09:13.597742-07,00:02:00,3
2012-01-19 07:49:00,dee37862-6869-488e-b301-c21bc69e6f1e,00000000-56ff-538b-2223-e1800b5e3ddb,fitbit,6011012:1,Fitbit,health-sleep,2012-01-19 07:49:00,2012-01-19 07:51:00,0.0,NaN,NaN,2017-10-11 11:07:13.318881-07,2017-10-11 17:32:01.100877-07,00:02:00,3


# Back End Script 

### We are writing a python script to generate a table of insights for a particular user. This script will exist in one function, we can define functions within the function, but we want to streamline as much as possible for maximum efficiency. 

#### Insights To Generate  
- steps by sleep 
- sleep by heartrate 
- 

 _id: UUID (primary key)

owner: UUID (reference to User)

source: STRING - Describes where this insight came from. For example, insights created manually from the admin dashboard will have their source set as admin.

category: STRING - indicates which category/vertical the insight belongs to

type: STRING - A short identifier unique for each type of insight. Example: An insight recommending the user to take x steps to improve sleep might be identified by a type such as "step-count:sleep:achieve-steps".

message: TEXT - A textual representation of the insight, to be used in places where we can't display rich media, such as notifications. Example: "Get 2500 steps in by noon for a more restful sleep".

data: JSON - a JSON blob with detailed info on the insight. We make this a JSON field so that it can be easily extended in the future with new types of insights, without requiring changes to the DB schema.

upvote: BOOLEAN - A boolean field that is set to true when the user marks this insight as "useful" or gives it a "thumbs up". Default to false.

downvote: BOOLEAN - A boolean field that is set to true when the user marks this insight as "not useful" or gives it a "thumbs down". Default to false.


For the initial version, the JSON structure of the data field will be relatively simple:

{
    "message": "Get 2500 steps in by noon for a more restful sleep",
    "metrics": ["step-count", "sleep"],
    "explanation": "Science shows that getting physical activity and sunlight in the morning can help to keep your circadian rhythm in optimal shape. Your circadian rhythm is..."
}

In [3]:
#This is what the table is going to look like. 
#There are nine categories. Above is a short description of each column. 
UserInsight = pd.DataFrame(columns=['_id','owner','source','category','type','message','data','upvote','downvote'])
UserInsight

,_id,owner,source,category,type,message,data,upvote,downvote


In [4]:
#Example of how to insert into the table. 

blob = {"message":"If you keep losing weight","metrics":'The correlated variables',"explanation":'Leave this'}
example = ['000','John','Apple Watch','HeartRate','health-weight:heart-rate:positive','If you keep losing weight, your heart rate will drop',blob,'N/A','N/A']

UserInsight.loc[len(UserInsight)] = example
UserInsight

,_id,owner,source,category,type,message,data,upvote,downvote
0,000,John,Apple Watch,HeartRate,health-weight:heart-rate:positive,"If you keep losing weight, your heart rate wil...","{u'metrics': u'The correlated variables', u'me...",N/A,N/A


### Function  

In [18]:
#We need to tweak our functions to create an array like example above ^ 

class UserInsightFunction():
    
    def __init__(self, user_id, data_table):
        self.UserInsight = pd.DataFrame(columns=['_id','owner','source','category','type','message','data','upvote','downvote'])
        self.InsightTypes = []
        self.user_id = user_id
        self.data_table = data_table
        
    def prepare_table(self, x, y,time='D',data_sample=data_sample):
        steps = data_sample[data_sample['type'] == x]
        bodyfat = data_sample[data_sample['type'] == y]


        steps.index = steps['startDate']
        bodyfat.index = bodyfat['startDate']


        if x == "health-sleep":
            steps['value'] = pd.to_numeric(steps['duration'].dt.total_seconds())
            steps = steps.loc[:,'value']
            steps = steps.resample(time).mean()
        else:
            steps['value'] = pd.to_numeric(steps['value'])
            steps = steps.loc[:,'value']
            if x == 'health-step-count':
                steps = steps.resample(time).sum()
            else:
                steps = steps.resample(time).mean()


        bodyfat['value'] = pd.to_numeric(bodyfat['value'])
        bodyfat = bodyfat.loc[:,'value']
        if y == 'health-step-count':
            bodyfat = bodyfat.resample(time).sum()
        else:
            bodyfat = bodyfat.resample(time).mean()

        steps = steps.to_frame('first')
        steps['second'] = bodyfat

        return steps

    def corr_generator(self, steps):
        steps = steps.dropna()
        x = steps['first'].values
        y = steps['second'].values
        x = x.reshape(len(x), 1)
        y = y.reshape(len(x), 1)
        if not (x.shape == (0, 1) or y.shape == (0, 1)):
            regr = linear_model.LinearRegression()
            regr.fit(x, y)
            corr_info = stats.stats.pearsonr(x, y)
            r = corr_info[0][0]
            if corr_info[1] == 0.0:
                p = corr_info[1]
            else:
                p = corr_info[1][0]
            if (type(p) is np.ndarray):
                p = p[0]
            return r, p
        else:
            return 0, 1.1
    
    def CorrelationTable(self): 
        #create table of all correlations in our data set for the user.
        CorrelationTable = pd.DataFrame(columns=['Variable1','Variable2','Overall','Last 30 Days','Last 60 Days','Last 90 Days', 'p-Overall','p-Last 30 Days','p-Last 60 Days','p-Last 90 Days'])
        data = self.data_table[self.data_table['owner'] == self.user_id]
        types = [types for types in data.type.unique()]
        data30 = data.last('30D')
        data7 = data.last('60W')
        data1 = data.last('90D')
        for type1, type2 in itertools.permutations(types, 2):
            # print(type1 + " " + type2)
            r, p = self.corr_generator(self.prepare_table(type1, type2, data_sample=data))
            #print(type1 + " " + type2 + " " + str(r) + " total")
            r30, p30 = self.corr_generator(self.prepare_table(type1, type2, data_sample=data30))
            #print(type1 + " " + type2 + " " + str(r30) + " 30")
            r7, p7 = self.corr_generator(self.prepare_table(type1, type2, data_sample=data7))
            #print(type1 + " " + type2 + " " + str(r7) + " 7")
            r1, p1 = self.corr_generator(self.prepare_table(type1, type2, data_sample=data1))
            #print(type1 + " " + type2 + " " + str(r1) + " 1")
            curRow = {'Variable1': type1, 'Variable2': type2, 'Overall': r, 'Last 30 Days': r30, 'Last 60 Days': r7, 'Last 90 Days': r1, 'p-Overall': p, 'p-Last 30 Days': p30, 'p-Last 60 Days': p7, 'p-Last 90 Days': p1}
            CorrelationTable = CorrelationTable.append(curRow, ignore_index=True)
            
        return CorrelationTable
        
    
    def InsightCreator(self):
        #This is where we want to create the insight arrays from the correlation table, and append to UserInsights
        return
    
    



In [19]:
userinsights = UserInsightFunction('00000000-56ff-538b-2223-e1800b5e3ddb', data_sample)
corTable = userinsights.CorrelationTable()

In [20]:
corTable[corTable["Variable1"]=='health-sleep']

,Variable1,Variable2,Overall,Last 30 Days,Last 60 Days,Last 90 Days,p-Overall,p-Last 30 Days,p-Last 60 Days,p-Last 90 Days
0,health-sleep,health-bmi,0.344753,0.336899,0.260043,0.480703,2.584235e-04,0.158409,0.008634,0.004631
1,health-sleep,health-weight,0.375367,0.450142,0.309944,0.560825,3.297085e-05,0.031137,0.001039,0.000137
2,health-sleep,daily-summary:health-sleep,-0.367864,0.169758,-0.204230,-0.200448,3.573194e-05,0.474289,0.043680,0.241121
3,health-sleep,health-height,0.000000,0.000000,0.000000,0.000000,1.100000e+00,1.100000,1.100000,1.100000
4,health-sleep,health-fat-free-mass,-0.085489,0.256617,-0.036353,0.292035,4.100996e-01,0.375825,0.733730,0.131569
5,health-sleep,health-body-fat,0.068509,-0.086565,0.033816,0.150642,5.071802e-01,0.768566,0.750319,0.444174
6,health-sleep,health-fat-mass-weight,0.491912,0.020532,0.420346,0.255430,4.135781e-07,0.944459,0.000037,0.189563
7,health-sleep,health-heart-rate,0.039347,-0.302308,0.116663,-0.069896,6.492560e-01,0.141893,0.184508,0.589320
8,health-sleep,health-step-distance,0.101558,-0.133986,0.010925,0.113075,2.411752e-01,0.694499,0.902972,0.517796
9,health-sleep,health-step-count,-0.153748,-0.178688,-0.241730,0.022880,3.771037e-02,0.392768,0.002143,0.854188
